In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

### Zipfiles:

In [2]:
from zipfile import ZipFile

In [3]:
master = "MasterDictionary-20221105T045120Z-001.zip"
z = ZipFile(master)
positive = str(z.read('MasterDictionary/positive-words.txt'))
positive = positive.replace("\\n"," ")
positive = positive.split()
print("Positive words: " ,len(positive))

negative = str(z.read('MasterDictionary/negative-words.txt'))
negative = negative.replace("\\n"," ")
negative = negative.split()
print("Negative words: ",len(negative))

Positive words:  2007
Negative words:  4784


In [4]:
stop_words = "StopWords-20221105T045120Z-001.zip"
s = ZipFile(stop_words)
s.infolist()

stop01 = s.read('StopWords/StopWords_Generic.txt')
stop02 = s.read('StopWords/StopWords_Currencies.txt')
stop03 = s.read('StopWords/StopWords_Names.txt')
stop04 = s.read('StopWords/StopWords_Geographic.txt')
stop05 = s.read('StopWords/StopWords_GenericLong.txt')
stop06 = s.read('StopWords/StopWords_Auditor.txt')
stop07 = s.read('StopWords/StopWords_DatesandNumbers.txt')

stop = stop01+stop02+stop03+stop04+stop05+stop06+stop07

stop1 = stop.lower()
stop1 = str(stop1)
stop1 = stop1.replace("\\n","").replace("\\r"," ").replace("|"," ")
stop1 = stop1.split()
stop = stop1
print("Stop words: " ,len(stop))

Stop words:  14236


#### Data Frame to Store Output:

In [5]:
df = pd.DataFrame(columns=["URL_ID","URL","POSITIVE SCORE","NEGATIVE SCORE","POLARITY SCORE","SUBJECTIVITY SCORE","AVG SENTENCE LENGTH","PERCENTAGE OF COMPLEX WORDS","FOG INDEX","AVG NUMBER OF WORDS PER SENTENCE","COMPLEX WORD COUNT","WORD COUNT","SYLLABLE PER WORD","PERSONAL PRONOUNS","AVG WORD LENGTH"])

### Url Text Extraction:

In [6]:
dataframe1 = pd.read_excel('input.xlsx')
dataframe1.iloc[113,1]

'https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/'

In [7]:
x=0
u=0
e=37
while x<114:
    url = dataframe1.iloc[x,1]
    if x not in [7,20,107]:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',}

        r = requests.get(url, headers=headers)
        c = r.content

        soup = BeautifulSoup(c, "html.parser")
        title = soup.title.text.replace("- Blackcoffer Insights", "")
        title1 = title.lower()
        title2 = title1.split()

        para1 = soup.find(class_="td-post-content").get_text().replace("\n"," ").replace("\xa0", "")
        para2 = para1.replace(".","").replace(',','').replace("("," ").replace(")"," ")
        para3 = para2.lower()
        para4 = title2+para3.split()
        #print(len(para4))

        para_join = " ".join(para4)
        #para_join

        ######### TO SAVE EXTRACTED ARTICLE IN TEXT FILE WITH UR_ID AS ITS NAME #########
        text_file = open("C:/Users/drkdr/OneDrive/Desktop/Test Assignment (Black Coffer)/"+str(e)+".txt","w")

        #write string to file
        ti = title.split()
        pa = para2.split()
        para_join_text = ti+pa
        normal_string=" ".join(ch for ch in para_join_text if ch.isalnum())
        text_file.write(normal_string)
        #close file
        text_file.close()
        #-------------------------------------------------------------------------------#

        para = word_tokenize(para_join)

        para_t= para.copy()
        #print("Total length: ",len(para_t))
        i=0
        while i < len(para_t)-1:
            if para_t[i] in stop:
                para_t.pop(i)
            i+=1
        paraf = para_t
        #print("Length after excluding stop words: ",len(paraf))

        words={}
        i=0
        while i < len(paraf)-1:
            if paraf[i] in positive:
                words[paraf[i]]="Positive"
            elif paraf[i] in negative:
                words[paraf[i]]="Negative"
            i+=1
        #print("Positive and Negative words in Para: ", len(words))

        #1 - EXTRACTING DERIVED VARIABLES::
        #-------------------------------------------------------------------------------#
        p=0
        n=0
        for i in words:
            if words[i]=="Positive":
                p+=1
            else:
                n+=1
        #print("2 - Positive Score: ", p)
        #print("3 - Negative Score: ", n)

        #Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
        #Ranges from -1 to +1

        pol = (p-n)/((p+n)+0.000001)
        #print("4 - Polarity Score: ", pol)

        #Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
        #Ranges from 0 to 1

        sub = (p+n)/((len(paraf))+0.000001)
        #print("5 - Subjectivity Score: ", sub)
        #-------------------------------------------------------------------------------#


        #2 - ANALYSIS OF READABILITY
        #-------------------------------------------------------------------------------#
        def count_syllables(word):
            word = word.lower()
            counter = 0
            is_previous_vowel = False
            if word[len(word)-2]=="e" and word[len(word)-1]=="d":
                word=word.replace("ed","")
            elif word[len(word)-2]=="e" and word[len(word)-1]=="s":
                word=word.replace("es","")
            for index, value in enumerate(word):
                if value in ["a", "e", "i", "o", "u", "y"]:
                    if index == len(word) - 1:
                        if value == "e":
                            if counter == 0:
                                counter += 1
                        else:
                            counter += 1
                    else:
                        if is_previous_vowel == True:
                            counter += 1
                            is_previous_vowel = False
                            break
                    is_previous_vowel = True
                else:
                    if is_previous_vowel == True:
                        counter += 1
                    is_previous_vowel = False
            return counter

        #-------------------------------------------------------------------------------#
        para_test = paraf.copy()
        i=0
        complex_words=[]
        syllables = []
        while i < len(para_test):
            b=count_syllables(para_test[i])
            syllables.append(b)
            if b>2:
                complex_words.append(para_test[i])
            i+=1
        len(syllables)
        #-------------------------------------------------------------------------------#


        #4 - COMPLEX WORD COUNT::
        #-------------------------------------------------------------------------------#
        #print("Complex Word Count: ",len(complex_words))


        para_sent = sent_tokenize(para1)

        avg_sentence_length = len(para)/len(para_sent)
        #print("6 - Average Sentence Length: ", avg_sentence_length)

        complex_per = (len(complex_words)/len(para_test))*100
        #print("7 - Percentage of complex words: ", complex_per)

        fog_index = 0.4*(avg_sentence_length + complex_per)
        #print("8 - Fog Index : ",fog_index)
        #-------------------------------------------------------------------------------#


        #5 - WORD COUNT::
        #-------------------------------------------------------------------------------#
        text = para3
        text_tokens = word_tokenize(text)
        word_c = [word for word in text_tokens if not word in stopwords.words()]

        #print("Removed all the stop words using stopwords class of nltk: Length=> ", len(word_c))

        i=0
        word_count=[]
        while i < len(word_c):
                if word_c[i]=="?":
                    pass
                elif word_c[i]==".":
                    pass
                elif word_c[i]==",":
                    pass
                elif word_c[i]=="!":
                    pass
                elif len(word_c[i])==1 and word_c[i] not in ["a","i"]:
                    pass
                else:
                    word_count.append(word_c[i])
                i+=1

        #print("After removing punctuations like (? ! , .) : Length => ", len(word_count))
        #print("Word Count: ", len(word_count))
        #-------------------------------------------------------------------------------#


        #6 - SYLLABLE COUNT PER WORD ::
        #-------------------------------------------------------------------------------#

        syllables1 = dict(zip(para_test,syllables))
        syllables2 = sum(syllables)/len(word_count)
        #print("Syllable per word : ", syllables2)
        #print("Syllable Count Per Word : ","\n", syllables1)


        #-------------------------------------------------------------------------------#


        #7- COUNTING PESRSONAL PRONOUNS :: ( “I,” “we,” “my,” “ours,” and “us” )
        #-------------------------------------------------------------------------------#
        import re
        t = para3
        pattern = re.compile('I')
        matches = pattern.finditer(t)
        I_count=0
        for match in matches:
            I_count+=1

        pattern = re.compile('we')
        matches = pattern.finditer(t)
        we_count=0
        for match in matches:
            we_count+=1

        pattern = re.compile('my')
        matches = pattern.finditer(t)
        my_count=0
        for match in matches:
            my_count+=1

        pattern = re.compile('ours')
        matches = pattern.finditer(t)
        ours_count=0
        for match in matches:
            ours_count+=1

        pattern = re.compile('us')
        matches = pattern.finditer(t)
        us_count=0
        for match in matches:
            us_count+=1
        #
        total_pronouns = I_count+we_count+my_count+ours_count+us_count
        #print("Personal Pronouns: ","|I:",I_count,"|we:",we_count,"|my:",my_count,"|ours:", ours_count,"|us:",us_count,"|")
        #-------------------------------------------------------------------------------#


        #8 - AVERAGE WORD LENGTH::
        #-------------------------------------------------------------------------------#
        text_s = word_c
        a=0
        for i in text_s:
            a=a+len(i)

        #print("Sum of all characters in each word : ", a)

        #print("Total number of words: ", len(word_c))

        avg_word_length = a/len(word_c)

        #print("Average Word Length : ", avg_word_length)

        avg_words_per_sent = len(word_c)/len(para_sent)
        #print("9 - Average Words Per Sentence : ", avg_words_per_sent)
        #-------------------------------------------------------------------------------#

        complex_words1 = len(complex_words)
        word_count1 = len(word_count)

        #print("\n")
        #print("2  - POSITIVE SCORE: ", p)
        #print("3  - NEGATIVE SCORE: ", n)
        #print("4  - POLARITY SCORE: ", pol)
        #print("5  - SUBJECTIVITY SCORE: ", sub)
        #print("6  - AVERAGE SENTENCE LENGTH: ", avg_sentence_length)
        #print("7  - PERCENTAGE OF COMPLEX WORDS: ", complex_per)
        #print("8  - FOG INDEX : ",fog_index)
        #print("9  - AVG NUMBER OF WORDS PER SENTENCE : ", avg_words_per_sent)
        #print("10 - COMPLEX WORD COUNT : ", complex_words1)
        #print("11 - WORD COUNT : ", word_count1)
        #print("12 - SYLLABLE PER WORD : ", syllables2)
        #print("13 - PERSONAL PRONOUNS : ", total_pronouns)
        #print("14 - AVG WORD LENGTH : ", avg_word_length)


        list_variables = [e, url, p, n, pol, sub, avg_sentence_length, complex_per, fog_index, avg_words_per_sent, complex_words1, word_count1, syllables2, total_pronouns, avg_word_length]
        #print(list_variables)

        df.loc[u]=list_variables

    else:
        list_na = [e,url,"Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL","Invalid URL",]
        df.loc[u]=list_na
    
    print(x," of 113")
    u+=1
    e+=1
    x+=1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


In [8]:
pd.set_option('display.max_rows', None)

In [11]:
df.head(5)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,46,25,0.295775,0.058101,23.74026,29.214403,21.181865,13.246753,357,999,2.393393,56,7.530392
1,38,https://insights.blackcoffer.com/what-if-the-c...,39,29,0.147059,0.07489,18.4375,17.84141,14.511564,7.6625,162,576,2.579861,36,6.76509
2,39,https://insights.blackcoffer.com/what-jobs-wil...,48,25,0.315068,0.064716,20.529412,27.659574,19.275594,10.411765,312,859,2.563446,61,7.450847
3,40,https://insights.blackcoffer.com/will-machine-...,37,22,0.254237,0.056405,17.8,17.686424,14.19457,7.568421,185,689,2.516691,60,6.783032
4,41,https://insights.blackcoffer.com/will-ai-repla...,37,25,0.193548,0.053037,23.088608,21.12917,17.687111,11.113924,247,820,2.454878,65,6.817768


In [10]:
df.to_excel("Final Output Submission.xlsx", index=False)